#### Descargando la data 

Descargando los datos para el entrenamiento de los modelos.

#### Módulos

In [14]:
# Módulos
import ee
import folium
import geehydro

In [15]:
# Inicio de API
ee.Initialize()

#### Función generateGreedForGeometry

In [16]:
#funcion para la generacion de grillas
def generateGreedForGeometry(geom,dx,dy):
    """
    Función para generar grillas partiendo de una zona geográfica.
    """

    bounds = ee.Geometry(geom).bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    xmin = ll.get(0)
    xmax = ur.get(0)
    ymin = ll.get(1)
    ymax = ur.get(1)

    xx = ee.List.sequence(xmin, xmax, dx)
    yy = ee.List.sequence(ymin, ymax, dy)

    #************************************
    def contYYa(y,x,dx,dy):

        x1 = ee.Number(x)
        x2 = ee.Number(x).add(ee.Number(dx))
        y1 = ee.Number(y)
        y2 = ee.Number(y).add(ee.Number(dy))

        coords = ee.List([x1, y1, x2, y2])
        rect = ee.Algorithms.GeometryConstructors.Rectangle(coords)

        return ee.Feature(rect)

    def contYY(x):
        return yy.map(lambda yi: contYYa(y=yi,x=x,dx=dx,dy=dy) )
    #************************************

    cells = xx.map(contYY).flatten() 

    return ee.FeatureCollection(cells).filterBounds(geom)#.map(lambda c: ee.Feature(c).intersection(geom) )

#### Ejemplo

In [17]:
#----------------------------------------------
# Seleccion del parque
#----------------------------------------------

# region
polygonCollection = 'WCMC/WDPA/current/polygons'
polygonName = 'Cerro Saroche'#'Canaima National Park'#'Guatopo'

# poligono
polygon = ee.FeatureCollection(polygonCollection) \
            .filter(ee.Filter.eq('NAME', polygonName))


# geometria
state = polygon.geometry()
dx = 0.10#0.0288
dy = 0.10#0.0288

# creando grilla
greed = generateGreedForGeometry(state,dx,dy)
centroide = state.centroid().coordinates().getInfo()[::-1]

def getCentroid(feature):
    keepProperties = ['name', 'huc6', 'tnmid', 'areasqkm']
    centroid = feature.geometry().centroid()
    return ee.Feature(centroid).copyProperties(feature, keepProperties)

#centroides
centroids = greed.map(getCentroid)

print('área del parque Km2: ', 0.1*polygon.first().geometry().area().divide(100*100).getInfo() ) 
print('área de cuadricula Km2: ', 0.1*greed.first().geometry().area().divide(100*100).getInfo() )
print('número de centroides: ', centroids.size().getInfo())

área del parque Km2:  3629.434345090513
área de cuadricula Km2:  1216.8458265151064
número de centroides:  11


#### data ECMWF/ERA5/MONTHLY

In [5]:
#https://confluence.ecmwf.int/display/CKB/ERA5%3A+What+is+the+spatial+reference#ERA5:Whatisthespatialreference-5Gridgeometry
import numpy as np
from shapely.geometry import MultiLineString, MultiPoint, Polygon
from shapely.ops import polygonize

dx = 0.10
dy = 0.10

# generando grilla
x = np.arange(-180.0 + dx, 180.0 + dx, dx)
y = np.arange(-90.0 + dy, 90.0 + dy, dy)

hlines = [((x1, yi), (x2, yi)) for x1, x2 in zip(x[:-1], x[1:]) for yi in y]
vlines = [((xi, y1), (xi, y2)) for y1, y2 in zip(y[:-1], y[1:]) for xi in x]

# grillas
grids = list(polygonize(MultiLineString(hlines + vlines)))

# creando poligonops
google_poligon = ee.Geometry(polygon.geometry()).bounds().coordinates().get(0).getInfo()
poligono = Polygon(list(map(lambda x: tuple(x), google_poligon)))

# interseccion
intersection = list(map(lambda y: y if poligono.intersection(y) else None,grids))

# poligonos en area
poligono_ECMWF = [x for x in list(filter(None,intersection))]

# creando lista de centroides
lista_ = ee.List([np.array(x.centroid.coords).tolist()[0] for x in poligono_ECMWF])
centroides_ECMWF = ee.FeatureCollection(lista_.map( lambda x: ee.Feature(ee.Geometry.Point(x,proj='EPSG:4326'),{'id':lista_.indexOf(x)}) ))

In [ ]:
# Elevacion
elevation = ee.Image('CGIAR/SRTM90_V4').select('elevation')
slope = ee.Terrain.slope(elevation).clip(polygon)

# Precipitacion total
ECMWF = ee.ImageCollection("ECMWF/ERA5/MONTHLY")

# calculando el punto con la elevacion
reducer = ee.Reducer.median()
centroids_elevation = elevation.reduceRegions( ee.FeatureCollection(centroides_ECMWF), reducer.setOutputs(['elevation']), 90)

In [ ]:
def ECMWF_data(imagen_collection = ECMWF,
                date = '2015-12-01',
                imagen_elevation = centroids_elevation):

    reducer = ee.Reducer.median()
    centroids_ECMWF= imagen_collection\
                        .filterDate(date)\
                        .map(lambda x: x.reduceRegions(imagen_elevation,reducer.setOutputs(['total_precipitation']),2))

    lista = centroids_ECMWF.first().getInfo()

    if lista is None:
        return None

    return list(map(lambda x,y:
                    {'date': y,
                    'id':int(x.get('id')),
                    'longitude':x.get('geometry').get('coordinates')[0],
                    'latitude':x.get('geometry').get('coordinates')[1],
                    'dewpoint_2m_temperature': x.get('properties').get('dewpoint_2m_temperature'),
                    'elevation': x.get('properties').get('elevation'),
                    'maximum_2m_air_temperature': x.get('properties').get('maximum_2m_air_temperature'),
                    'mean_2m_air_temperature': x.get('properties').get('mean_2m_air_temperature'),
                    'mean_sea_level_pressure': x.get('properties').get('mean_sea_level_pressure'),
                    'minimum_2m_air_temperature': x.get('properties').get('minimum_2m_air_temperature'),
                    'surface_pressure': x.get('properties').get('surface_pressure'),
                    'total_precipitation': x.get('properties').get('total_precipitation'),
                    'u_component_of_wind_10m': x.get('properties').get('u_component_of_wind_10m'),
                    'v_component_of_wind_10m': x.get('properties').get('v_component_of_wind_10m')
                    },
                    lista.get('features'),
                    [lista.get('properties').get('system:index') for x in range(len(lista.get('features'))) ]
                    ))

#ECMWF_data()

In [ ]:
import pandas as pd
import numpy as np

# rango de meses
date_start="1970-01-01"
date_end="1989-12-01"

# consultando la data
data = np.array( list(filter(None,
                    list(map(lambda x: ECMWF_data(date=x)
                                    ,pd.date_range(start=date_start,end=date_end, closed='right',freq='M').strftime('%Y-%m').to_list()
                                    ))
                )))

data = data.reshape((
                    1,
                    data.shape[0]*data.shape[1]
                    )).tolist()[0]

data = pd.DataFrame(data)
data['year'] = pd.to_datetime(data['date'],format='%Y%m').dt.year
data['month'] = pd.to_datetime(data['date'],format='%Y%m').dt.month
data['date'] = data['date'].apply(int)
data['date_time'] = pd.to_datetime(data['date'],format='%Y%m')
data['polygonName'] = polygonName

data = data.sort_values(['date_time','id'],ascending=True)

data.head(5)

In [32]:
# save data pkl
data.to_pickle('./data/{polygonName}_data_1.pkl'.format(polygonName=polygonName))

In [33]:
# x and y given as array_like objects
import plotly.express as px

fig = px.scatter(data[data.date==201001],
y='latitude',
x='longitude',
color='total_precipitation',
title='Precipitación')
fig.show()

In [34]:
# mean NDVI in the Xingu Park
Map = folium.Map(location=centroide,zoom_start=11,control_scale =True,width='100%',height='100%')

Map.addLayer(slope, {min: 0, max: 60}, 'Slope')
Map.addLayer(greed,{'color':'#FFFFFF80'}, 'Grids')
Map.addLayer(centroids,{'color':'#000000'}, 'Centroids')
Map.addLayer(centroides_ECMWF,{'color':'#FF0000'}, 'centroides_ECMWF')

Map.setControlVisibility(layerControl=True,fullscreenControl=True,latLngPopup=True) # Layer control
Map